In [ ]:
# this didn't work. The grippers grabbed the plate but then dropped it. Had to reset and grippers got
# dropped into the trash. 
# I think it ws because grippers went slightly below the plate, probably because I'm using the wrong carrier
# I haven't found my carrier in the resource library. Neither deep well nor plate specific. 
# 2025 July 3 hjk

%load_ext autoreload
%autoreload 2

In [2]:
# Start by importing the LiquidHandler class, which will serve as a front end for all liquid handling operations.
# Backends serve as communicators between LiquidHandlers and the actual hardware. Since we are using a Hamilton STAR, we also import the STARBackend backend.

from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import STARBackend

In [3]:
# In addition, import the STARLetDeck, which represents the deck of the Hamilton STAR.
from pylabrobot.resources.hamilton import STARLetDeck

In [4]:
# Create a new liquid handler using STARBackend as its backend.
backend = STARBackend()
# backend = STARBackend(auto_detach_kernel_driver=True) 
# backend.auto_detach_kernel_driver = True

lh = LiquidHandler(backend=backend, deck=STARLetDeck())

In [5]:
# The final step is to open communication with the robot. This is done using the setup() method.
await lh.setup()

#About 45s to initialize

In [6]:
# Start by importing the relevant objects and variables from the PyLabRobot package. This notebook uses the following resources:
from pylabrobot.resources import (
  TIP_CAR_480_A00,
  PLT_CAR_L5AC_A00,
  AGenBio_96_wellplate_Ub_2200ul,
  HTF, #Tip Rack with 96 1000ul High Volume Tip with filter
)

In [7]:
# Then create a tip carrier named tip carrier, which will contain tip rack at all 5 positions. These positions can be accessed using tip_car[x], and are 0 indexed.
tip_car = TIP_CAR_480_A00(name="tip carrier")
tip_car[0] = HTF(name="tips_01")
# Use assign_child_resources() to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.
# In the rails parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.
lh.deck.assign_child_resource(tip_car, rails=1)

In [8]:
# Repeat this for the plates.
pc1 = PLT_CAR_L5AC_A00(name="plate carrier 1")
pc1[0] = AGenBio_96_wellplate_Ub_2200ul(name="plate_01")
lh.deck.assign_child_resource(pc1, rails=13)

pc2 = PLT_CAR_L5AC_A00(name="plate carrier 2") #empty
lh.deck.assign_child_resource(pc2, rails=19)

pc3 = PLT_CAR_L5AC_A00(name="plate carrier 3") #empty
lh.deck.assign_child_resource(pc3, rails=25)


In [9]:
lh.summary()


Rail  Resource                      Type           Coordinates (mm)
(-6)  ├── trash_core96              Trash          (-58.200, 106.000, 229.000)
      │
(1)   ├── tip carrier               TipCarrier     (100.000, 063.000, 100.000)
      │   ├── tips_01               TipRack        (106.200, 073.000, 214.950)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(13)  ├── plate carrier 1           PlateCarrier   (370.000, 063.000, 100.000)
      │   ├── plate_01              Plate          (374.000, 071.500, 182.350)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(19)  ├── plate carrier 2           PlateCarrier   (505.000, 063.000, 100.000)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(25)  ├── plate carrier 3           PlateCarrier   (640.000, 063.000, 100.000)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <em

In [10]:

# --- move the plate from site 0 → site 1 on the *same* carrier -------------
src  = pc1[0]          # the plate you want to move
dest = pc1[1]          # an empty site on that carrier

await lh.move_plate(                       # high-level helper
    src,
    dest,
    use_arm="core",                        # tell PLR to use the CO-RE gripper
    channel_1=2, channel_2=3,              # zero-based channels that hold the fingers
    core_grip_strength=30,                 # 0 = loose … 99 = very tight
    return_core_gripper=True               # park the fingers afterwards
)


STARFirmwareError: {'Pipetting channel 2': HardwareError('Stop during execution of command'), 'Pipetting channel 3': HardwareError('Y-drive movement error')}, C0ZRid0009er99/00 P202/37 P302/57

In [11]:
await lh.stop()

In [14]:
# 0. Fresh backend & LH
# backend = STARBackend(auto_detach_kernel_driver=True)   # same serial-port you always use
# lh      = LiquidHandler(backend=backend, deck=STARDeck())

await lh.setup()                 # reconnect

# 1. Tell PLR where the fingers are right now
#    Your description: attachments sit on channels 1 & 2 (0-based)
await lh.return_core_gripper(
    channel_1=1,     # ← where the *left* finger sits now
    channel_2=2,     # ← where the *right* finger sits now
    minimum_traverse_height_at_beginning_of_a_command=150,  # mm; clear everything
)

# 2. Wait for the head to park the fingers.  The function returns when done.


USB device already connected. Closing previous connection.


AttributeError: 'LiquidHandler' object has no attribute 'return_core_gripper'